In [ ]:
# 📦 Install required libraries (run once)
!pip install yfinance plotly ipywidgets openpyxl --quiet
!jupyter nbextension enable --py widgetsnbextension

# 📚 Imports
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio
pio.renderers.default = 'notebook'

# 🏦 Bank tickers
tickers = {
    "HDFC Bank": "HDFCBANK.NS",
    "ICICI Bank": "ICICIBANK.NS",
    "SBI": "SBIN.NS",
    "Axis Bank": "AXISBANK.NS",
    "Kotak Bank": "KOTAKBANK.NS"
}

# 📊 Fetch & calculate SMA signals
def get_sma_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date, interval='1d')

    if df.empty or len(df) < 60:
        return None

    df['SMA20'] = df['Close'].rolling(window=20).mean()
    df['SMA50'] = df['Close'].rolling(window=50).mean()
    df['Signal'] = 0
    df.loc[df.index[50:], 'Signal'] = np.where(df['SMA20'][50:] > df['SMA50'][50:], 1, 0)
    df['Position'] = df['Signal'].diff()

    return df

# 📈 Plot SMA strategy
def plot_sma(df, bank_name, chart_type='line'):
    fig = go.Figure()

    if chart_type == 'candlestick':
        fig.add_trace(go.Candlestick(
            x=df.index, open=df['Open'], high=df['High'],
            low=df['Low'], close=df['Close'], name='Candlestick'
        ))
    else:
        fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close Price', mode='lines', line=dict(color='black')))

    fig.add_trace(go.Scatter(x=df.index, y=df['SMA20'], name='SMA 20', mode='lines', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA50'], name='SMA 50', mode='lines', line=dict(color='orange')))

    buy = df[df['Position'] == 1]
    sell = df[df['Position'] == -1]

    fig.add_trace(go.Scatter(
        x=buy.index, y=buy['SMA20'], mode='markers', name='Buy Signal',
        marker=dict(color='green', size=10, symbol='triangle-up')
    ))

    fig.add_trace(go.Scatter(
        x=sell.index, y=sell['SMA20'], mode='markers', name='Sell Signal',
        marker=dict(color='red', size=10, symbol='triangle-down')
    ))

    fig.update_layout(
        title=f"SMA Strategy - {bank_name}",
        xaxis_title="Date", yaxis_title="Price (INR)",
        height=600, template="plotly_white", legend_title="Legend"
    )
    fig.show()

# 💹 Calculate Profit/Loss
def calculate_pnl(df):
    df_trades = df[df['Position'].isin([1, -1])].copy()
    df_trades['Price'] = df_trades['Close']
    df_trades = df_trades[['Price', 'Position']]
    
    pnl = 0
    positions = []
    
    for i in range(0, len(df_trades) - 1, 2):
        try:
            buy = df_trades.iloc[i]
            sell = df_trades.iloc[i + 1]
            if buy['Position'] == 1 and sell['Position'] == -1:
                trade_pnl = sell['Price'] - buy['Price']
                pnl += trade_pnl
                positions.append((buy.name.date(), sell.name.date(), trade_pnl))
        except:
            continue
    return pnl, positions

# 📁 Export to CSV/Excel
def export_signals(df, bank_name):
    signals = df[df['Position'].isin([1, -1])][['Close', 'SMA20', 'SMA50', 'Position']]
    signals.to_csv(f'{bank_name}_signals.csv')
    signals.to_excel(f'{bank_name}_signals.xlsx')
    print(f"✅ Signals saved as {bank_name}_signals.csv and .xlsx")

# 🎛️ UI Widgets
bank_dropdown = widgets.Dropdown(options=tickers, value='HDFCBANK.NS', description='Select Bank:')
chart_toggle = widgets.ToggleButtons(options=['line', 'candlestick'], description='Chart Type:')
start_picker = widgets.DatePicker(description='Start Date', value=(datetime.today() - timedelta(days=150)).date())
end_picker = widgets.DatePicker(description='End Date', value=datetime.today().date())
export_button = widgets.Button(description="Export Signals", button_style='info')
output = widgets.Output()

# 🔁 Event Handler
def on_update(change=None):
    output.clear_output(wait=True)
    with output:
        bank_name = [k for k, v in tickers.items() if v == bank_dropdown.value][0]
        start = start_picker.value.strftime('%Y-%m-%d')
        end = end_picker.value.strftime('%Y-%m-%d')
        df = get_sma_data(bank_dropdown.value, start, end)

        if df is not None:
            plot_sma(df, bank_name, chart_type=chart_toggle.value)
            pnl, trades = calculate_pnl(df)
            print(f"💰 Total P&L: ₹{pnl:.2f}")
            print("📅 Trades:")
            for b, s, p in trades:
                print(f"  Buy: {b} → Sell: {s} → ₹{p:.2f}")
            export_button.on_click(lambda b: export_signals(df, bank_name))
        else:
            print("❌ No sufficient data found.")

# 🔗 Link widgets to handler
bank_dropdown.observe(on_update, names='value')
chart_toggle.observe(on_update, names='value')
start_picker.observe(on_update, names='value')
end_picker.observe(on_update, names='value')

# 🧩 Display UI
display(widgets.VBox([bank_dropdown, chart_toggle, start_picker, end_picker, export_button, output]))
on_update()


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
